In [1]:
# Import necessary libraries
import argparse
from tmu.preprocessing.standard_binarizer.binarizer import StandardBinarizer
import numpy as np
import cv2
from skimage.feature import local_binary_pattern
from keras.datasets import cifar10
from tmu.models.classification.vanilla_classifier import TMClassifier
from time import time

# Load CIFAR-10 dataset
(X_train_org, Y_train), (X_test_org, Y_test) = cifar10.load_data()


2023-11-30 14:22:07.836411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 14:22:07.836485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 14:22:07.838112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 14:22:07.850299: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2023-11-30 14:22:08,673 - tensorflow - DEBUG - Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2023-11-30 14:22:08,863 - h5py._conv - DEBUG - Creating converter from 7 to 5
2023-11-30 14:22:08,864 - h5py._conv - DEBUG - Creating converter from 5 to 7
2023-11-30 14:22:08,865 - h5py._conv - DEBUG - Creating converter from 7 to 5
2023-11-30 14:22:08,865 - h5py._conv - DEBUG - Creating converter from 5 to 7


2023-11-30 14:22:08.801529: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2023-11-30 14:22:09,445 - numexpr.utils - INFO - Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2023-11-30 14:22:09,446 - numexpr.utils - INFO - Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-11-30 14:22:09,447 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Copy the datasets
X_train = np.copy(X_train_org)
X_test = np.copy(X_test_org)

# Reshape labels
Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])

# Apply Local Binary Pattern and binarize images
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[3]):
        lbp = local_binary_pattern(X_train[i, :, :, j], P=8, R=2, method="uniform")
        X_train[i, :, :, j] = np.where(lbp < np.mean(lbp), 0, 1)

for i in range(X_test.shape[0]):
    for j in range(X_test.shape[3]):
        lbp = local_binary_pattern(X_test[i, :, :, j], P=8, R=2, method="uniform")
        X_test[i, :, :, j] = np.where(lbp < np.mean(lbp), 0, 1)


In [3]:
# Other parameters
T = 600
s = 4.0
num_clauses = 2000
max_included_literals = 32
device = "GPU"
weighted_clauses = True
epochs = 100
patch_size = 7
type_i_ii_ratio = 1.0

# Initialize TM Classifier with the chosen set of hyperparameters
tm = TMClassifier(
    number_of_clauses=num_clauses,
    T=T,
    s=s,
    max_included_literals=max_included_literals,
    platform=device,
    weighted_clauses=weighted_clauses,
    type_i_ii_ratio=type_i_ii_ratio,
    patch_dim=(patch_size, patch_size)
)

# Define the batch size
batch_size = 64  # Adjust based on your system's capabilities

# Function to yield batches from the dataset
def get_batches(X, Y, batch_size):
    for start in range(0, len(X), batch_size):
        end = start + batch_size
        yield X[start:end], Y[start:end]

# Train the TM Classifier with batch processing
for epoch in range(epochs):
    start_training = time()
    
    for X_batch, Y_batch in get_batches(X_train, Y_train, batch_size):
        tm.fit(X_batch, Y_batch)
    
    stop_training = time()

    # Evaluate the TM Classifier
    start_testing = time()
    Y_test_predicted, Y_test_scores = tm.predict(X_test, return_class_sums=True)
    stop_testing = time()
    result_test = 100 * (Y_test_scores.argmax(axis=1) == Y_test).mean()

    # Print accuracy for the current epoch
    print(f"Epoch {epoch+1} - Accuracy: {result_test:.2f}% - Training Time: {stop_training - start_training:.2f}s - Testing Time: {stop_testing - start_testing:.2f}s")
    np.savetxt("CIFAR10LBP_%d_%d_%d_%.1f_%d_%d_%d.txt" % (epoch+1, num_clauses, T, s, patch_size, max_included_literals, weighted_clauses), Y_test_scores, delimiter=',')


2023-11-30 14:22:48,161 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/64ad9a98c2545560a4dc31bd3a8dd0db9a262c49.ptx'.
2023-11-30 14:22:48,164 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/13fc2c8820318bc333016418ed940d81b88b1141.ptx'.
2023-11-30 14:22:48,166 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/5e60a7bb0bce1763c74eb0e200eea1e1b001d05e.ptx'.
2023-11-30 14:22:48,168 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/26b1d638292d278547515d52dcd4750656fa6a05.ptx'.
2023-11-30 14:22:48,419 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/64ad9a98c2545560a4dc31bd3a8dd0db9a262c49.ptx'.
2023-11-30 14:22:48,421 - tmu.clause_bank.clause_bank_cuda - INFO - Loading compiled CUDA module from '/tmp/tm_kernels/13fc2c8820318bc333016418ed940d81b88b1141.ptx'.
2023

KeyboardInterrupt: 